In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import dlib
import cv2
import os
%matplotlib inline

In [2]:
classes = ['chandrma', 'dhakshin dhruv', 'viganik']

In [3]:
model = tf.keras.models.load_model('current model1')

In [4]:
path_of_shape_predictor = 'D:\\mtech\\implementation\\dlib_model\\shape_predictor_68_face_landmarks_GTX.dat'

predictor = dlib.shape_predictor(path_of_shape_predictor)
detector = dlib.get_frontal_face_detector()


In [6]:
cap = cv2.VideoCapture('D:\\mtech\\implementation\\hindi_3_word_dataset\\chandrma\\01.mp4')
seq = []
while True:
    ret,frame = cap.read()
    if not ret:
        break
    resized = cv2.resize(frame,(256,512))
    gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
    results = detector(gray)
    for face in results:
        landmarks = predictor(gray, face)
        lips_points = []
        for i in range(48, 68):  
            x, y = landmarks.part(i).x, landmarks.part(i).y
            lips_points.append((x, y))
        x, y, w, h = cv2.boundingRect(np.array(lips_points))
        cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cropped = resized[y:y+h,x:x+w]
        seq.append(cv2.resize(cropped,(128,64)))
        seq = seq[-30:]
        if len(seq)== 30:
            prediction = model.predict(np.expand_dims(seq,axis=0))
            if prediction[0][np.argmax(prediction)]>0.8:
                text = classes[np.argmax(model.predict(np.expand_dims(seq,axis=0)))]
                position = (10, 50)  
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 1
                font_color = (0, 255, 0) 
                font_thickness = 2
                cv2.putText(gray, text, position, font, font_scale, font_color, font_thickness)
        cv2.imshow("video",gray)

    if cv2.waitKey(150)&0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()     

1/1 [==============================] - 0s 86ms/step
